# Simulating and fitting data using a simple Stick model

As in the previous tutorial we first create an acquisition scheme that we will use to simulate and fit data.

In [1]:
# load the necessary modules
from microstruktur.signal_models import cylinder_models, gaussian_models
from microstruktur.core import modeling_framework
from microstruktur.acquisition_scheme.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

acquisition_path = modeling_framework.GRADIENT_TABLES_PATH
bvalues_SI = np.loadtxt(join(acquisition_path, 'bvals_hcp_wu_minn.txt')) * 1e6  # in s/m^2
gradient_directions = np.loadtxt(join(acquisition_path, 'bvecs_hcp_wu_minn.txt'))  # on unit sphere
delta = 0.0106 # in seconds
Delta = 0.0431 # in seconds
acq_scheme = acquisition_scheme_from_bvalues(
    bvalues_SI, gradient_directions, delta, Delta)

Using this acquisition scheme, we will simulate data using a simple Stick model, and then use the same Stick model to fit the signal again. The first thing we have to do is instantiate the model we need.

NOTE: this example is EXACTLY the same with any of the other models, you only need to change the model and appropriate input parameters.

In [3]:
stick = cylinder_models.C1Stick()

To find more information on the model parameters check its function documentation. We can also print the parameter cardinality to figure out the parameter names, and their input format.

In [4]:
stick.parameter_cardinality

OrderedDict([('lambda_par', 1), ('mu', 2)])

As the function shows, we need to give the Stick one parallel diffusivity as lambda_par, and its orientation mu as angles on the sphere [theta, phi].  
For the example we align the Stick with some angle and give it a diffusivity of 1.7e-9 m^2/s. We obtain the right ordering for the input of the function by using the model's parameters_to_parameter_vector() function:

In [5]:
mu = (np.pi / 2., np.pi / 2.)
lambda_par = 1.7e-9
parameter_vector = stick.parameters_to_parameter_vector(
    lambda_par=lambda_par, mu=mu)

A diffusion signal, for the given model parameters and our previously made acquisition scheme is then simply generated as follows:

In [6]:
E = stick.simulate_signal(acq_scheme, parameter_vector)

Let's assume this signal is now unknown, and we want to fit the Stick model to this signal to find best fitting model parameters. Using the same procedure as before, we now set an initial condition for the same parameters from which we start the optimization.

In [7]:
initial_mu = np.random.rand(2)
initial_lambda_par = np.random.rand() * 1e-9
x0 = stick.parameters_to_parameter_vector(
    lambda_par=initial_lambda_par, mu=initial_mu)

Using the data, acquisition scheme and initial guess, we fit the model using the following one-liner. We can see the correct model parameters are obtained.

In [8]:
res = stick.fit(E, acq_scheme, x0)
print 'Initial guess:   ', x0
print 'Optimized result:', res
print 'Ground truth:    ', parameter_vector

Initial guess:    [  7.32749512e-10   5.76107626e-01   3.71209752e-01]
Optimized result: [  1.69999638e-09   1.57079580e+00   1.57079629e+00]
Ground truth:     [  1.70000000e-09   1.57079633e+00   1.57079633e+00]


The names of the parameters can again be obtained using parameter_vector_to_parameters.

In [9]:
stick.parameter_vector_to_parameters(res)

{'lambda_par': array([  1.69999638e-09]),
 'mu': array([ 1.5707958 ,  1.57079629])}